In [1]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPool2D, Conv2D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm


IMPORT DATA:

Import the ASL data set from the location where the file is downloaded

In [2]:
Data_Dir = 'F:\study\sem 10\cvpr\VGG 16\dataset'
TEST_DATA_DIR = os.path.join(Data_Dir, 'asl_alphabet_test/asl_alphabet_test')
TRAIN_DATA_DIR = os.path.join(Data_Dir, 'asl_alphabet_train/asl_alphabet_train')

EXTRACT Train Data:

Extract all the train data in CATEGORIES variable

In [3]:
CATEGORIES = []

for i in os.listdir(TRAIN_DATA_DIR):
    CATEGORIES.append(i)
    
print(CATEGORIES)

['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


PRE PROCESS DATA:

Pre prosessing of all data is done here

In [4]:
images = []
labels = []
img_size = 32,32

for c in CATEGORIES:
    path = os.path.join(TRAIN_DATA_DIR, c) 
    class_num = CATEGORIES.index(c) 
    for img in tqdm(os.listdir(path)):
        try:
            img_array = cv2.imread(os.path.join(path, img))   
            img_resized = cv2.resize(img_array, img_size)  
            images.append(img_resized) 
            labels.append(class_num)
        except WException as e:
            pass


100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:20<00:00, 145.30it/s]


In [5]:
images = np.array(images)
images = images.astype('float32')/255.0
labels = utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1)
print(labels.shape)

(87000, 29)


VGG 16 Architecture:

In [6]:
model = keras.Sequential([
    keras.Input(shape=(32,32,3)),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(29, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 256)         2

Compile the model and set the learnimg rate here and show the model summary and after that train the model

In [7]:
learning_rate = 0.0001
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
h = model.fit(x_train, y_train, batch_size=128, epochs=6, validation_split=0.1)

Epoch 1/6
551/551 [==============================] - 37s 47ms/step - loss: 2.8940 - accuracy: 0.1212 - val_loss: 0.9019 - val_accuracy: 0.6591
Epoch 2/6
551/551 [==============================] - 24s 43ms/step - loss: 0.6784 - accuracy: 0.7506 - val_loss: 0.2514 - val_accuracy: 0.9080
Epoch 3/6
551/551 [==============================] - 24s 43ms/step - loss: 0.2002 - accuracy: 0.9279 - val_loss: 0.0986 - val_accuracy: 0.9673
Epoch 4/6
551/551 [==============================] - 24s 43ms/step - loss: 0.1136 - accuracy: 0.9608 - val_loss: 0.1273 - val_accuracy: 0.9557
Epoch 5/6
551/551 [==============================] - 24s 43ms/step - loss: 0.0779 - accuracy: 0.9733 - val_loss: 0.0751 - val_accuracy: 0.9762
Epoch 6/6
551/551 [==============================] - 24s 43ms/step - loss: 0.0642 - accuracy: 0.9799 - val_loss: 0.0352 - val_accuracy: 0.9883


As you can see we done the training seisson here with 6 epochs it reached 97% of training accuracy and 98% of the validation accuracy which look pretty decent and it works good 